<a href="https://colab.research.google.com/github/pavansai26/outliers-dtection-using-elliptical-envelope-and-oneclass-svm/blob/master/outliers_dtection_using_elliptical_envelope_and_oneclass_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pylab
import numpy as np
import pandas as pd
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
pylab.rcParams.update({'font.size':14})

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
df=pd.read_csv('/gdrive/My Drive/Colab Notebooks/conn250K.csv')

In [0]:
df

In [0]:
df.shape

In [0]:
df.keys

In [0]:
df.columns

In [0]:
df.head()

In [0]:
df.tail()

In [0]:
df.head(0)

In [0]:
df.describe()

In [0]:
df.count()

In [0]:
df.isnull().sum()

In [0]:
qv1=df.iloc[:,0].quantile(0.25) # get quantile values and  iqr for outlier limits

In [0]:
qv1

In [0]:
qv2=df.iloc[:,0].quantile(0.5)
qv2

In [0]:
qv3=df.iloc[:,0].quantile(0.75)
qv3

In [0]:
qv_limit=1.5*(qv3-qv1)
qv_limit

In [0]:
un_outliers_mask=(df.iloc[:,0]>qv3+qv_limit) | (df.iloc[:,0]<qv1-qv_limit)
print(un_outliers_mask[:20])

In [0]:
un_outliers_data=df.iloc[:,0][un_outliers_mask]
print(un_outliers_data[:50])

In [0]:
fig=pylab.figure(figsize=(6,6))
ax=fig.add_subplot(1,1,1)
for name,y in zip(un_outliers_mask,un_outliers_data):
  ax.text(1,y,name)
ax.boxplot(df.iloc[:,0])
#ax.set_ylabel(df.iloc[:,0])

bivariate outliers:
create a scatterplot with an ellipse as a boundary for outliers.

In [0]:
params=[df.iloc[:,0],df.iloc[:,1]]
params_data=np.array([df.iloc[:,0],df.iloc[:,1]]).T

compute the elliptical envelope

In [0]:
ee=EllipticEnvelope()
ee.fit(params_data)

In [0]:
biv_outliers_mask=ee.predict(params_data)== -1

In [0]:
(biv_outliers_mask)

In [0]:
biv_outliers_data=params_data[biv_outliers_mask]


In [0]:
biv_outliers_data

In [0]:
xx,yy=np.meshgrid(np.linspace(params_data[:,0].min(),params_data[:,0].max(),100),np.linspace(params_data[:,1].min(),params_data[:,1].max(),100))

In [0]:
zz=ee.decision_function(np.c_[xx.ravel(),yy.ravel()])
zz=zz.reshape(xx.shape)

draw the scatterplot with the elliptical envelope and label the outliers

In [0]:
fig=pylab.figure(figsize=(10,10))
ax=fig.add_subplot(1,1,1)
for name,xy in zip(biv_outliers_mask[:50],biv_outliers_data[:50]):
  ax.text(xy[0],xy[1],name)
ax.contour(xx,yy,zz,levels=[0],linewidth=2)
ax.scatter(params_data[:,0],params_data[:,1],color='black')

ellipse is obtained with the data outside the ellipse data points called outliers.
but ellipse is not visible.

multivariate outliers

use the one class svm algorithm to classify unusual cases.

In [0]:
ocsvm=OneClassSVM(nu=0.25,gamma=0.05)
ocsvm.fit(df)



In [0]:
ocsvm.predict(df)==-1